In [70]:
from google.colab import drive 
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras import optimizers
from keras import layers
from keras import models

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K


base_dir = r'/content/gdrive/My Drive/MVP Datensatz/'
train_dir = r'/content/gdrive/My Drive/MVP Datensatz/Trainingsdatensatz5'


In [0]:
## create base_model

from keras.applications import Xception
base_model = Xception(
  weights = 'imagenet',
  include_top = False,
  pooling = 'avg', 
  input_shape = (299,299,3) # default vaules 
 )

In [78]:
## create train and validation generators

batch_s = 10  # global batch_size

## Train und Vaild Splitting

# Beispiel für mögliche Data Augmentations
train_datagen = ImageDataGenerator(featurewise_center = False,
                          samplewise_center = False,
                          featurewise_std_normalization = False,
                          samplewise_std_normalization = False,
                          rotation_range = 359,
                          width_shift_range = 30.0,
                          height_shift_range = 0.0,
                          brightness_range = None,
                          channel_shift_range = 50.0,
                          horizontal_flip = True, 
                          vertical_flip = True, 
                          rescale = 1./255,
                          validation_split = 0.2, 
                          interpolation_order = 1, 
                          dtype = 'float32')


valid_datagen = ImageDataGenerator(
                          rescale=1./255,
                          validation_split=0.2)

train_generator = valid_datagen.flow_from_directory(
    train_dir, 
    target_size = (299,299),
    batch_size = batch_s,
    class_mode = 'categorical',
    subset = 'training'
    )

validation_generator = valid_datagen.flow_from_directory(
    train_dir, 
    target_size = (299,299),
    batch_size = batch_s,
    class_mode = 'categorical',
    subset = 'validation'
)           

Found 80 images belonging to 5 classes.
Found 20 images belonging to 5 classes.


In [0]:
## extract features from data set

batch_size = batch_s
def extract_features(directory, sample_count, gen):
  features = np.zeros(shape=(sample_count, 2048)) 
  labels = np.zeros(shape=(sample_count, 5))
  generator = gen
  i=0
  for inputs_batch, labels_batch in generator:
    features_batch = base_model.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break
  return features, labels

train_features, train_labels = extract_features(train_dir, 80, train_generator)
validation_features, validation_labels = extract_features(train_dir, 20, validation_generator)


In [0]:
## create model

model = models.Sequential()
model.add(base_model)
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5,activation = 'softmax'))


In [81]:
# first: train only the top layers (which were randomly initialized)
# freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(
    train_generator,
    epochs = 10,
    validation_data = validation_generator
    )

Epoch 1/10
8/8 [==============================] - 3s 406ms/step - loss: 1.6038 - acc: 0.2375 - val_loss: 1.3833 - val_acc: 0.7000
Epoch 2/10
8/8 [==============================] - 1s 92ms/step - loss: 1.3063 - acc: 0.5125 - val_loss: 1.2329 - val_acc: 0.6500
Epoch 3/10
8/8 [==============================] - 1s 130ms/step - loss: 1.1668 - acc: 0.6000 - val_loss: 1.3483 - val_acc: 0.6000
Epoch 4/10
8/8 [==============================] - 1s 124ms/step - loss: 0.9353 - acc: 0.8000 - val_loss: 1.2151 - val_acc: 0.6000
Epoch 5/10
8/8 [==============================] - 1s 125ms/step - loss: 0.8244 - acc: 0.8250 - val_loss: 1.3226 - val_acc: 0.5500
Epoch 6/10
8/8 [==============================] - 1s 124ms/step - loss: 0.7629 - acc: 0.8250 - val_loss: 1.2610 - val_acc: 0.6500
Epoch 7/10
8/8 [==============================] - 1s 131ms/step - loss: 0.7707 - acc: 0.8000 - val_loss: 1.2246 - val_acc: 0.7000
Epoch 8/10
8/8 [==============================] - 1s 135ms/step - loss: 0.6799 - acc: 0.825

In [84]:
testbild = base_dir + r'/Testbilder/P1040695.JPG'
testbild = image.load_img(testbild, target_size=(299,299))
testbild = image.img_to_array(img)
testbild = np.expand_dims(testbild, axis = 0)


model.predict_classes(testbild)

ValueError: ignored

In [64]:
## Validdierung auf Abfragebildern
folder_path = r'/content/gdrive/My Drive/MVP Datensatz/Testbilder'
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(299, 299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

images = np.vstack(images)
classes = model.predict_classes(images,batch_s)
print(classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [0]:
model.save("/content/gdrive/My Drive/Keras Models/Xception_5-1.h5")
